In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

%matplotlib inline

In [ ]:
def plot_history(history, figsize=(12, 4), loss_train_key='loss', loss_val_key='val_loss', metric_train_key='accuracy', metric_val_key='val_accuracy'):
    loss_train = history.get(loss_train_key, [])
    loss_valid = history.get(loss_val_key, [])
    metric_train = history.get(metric_train_key, [])
    metric_valid = history.get(metric_val_key, [])

    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1)
    plt.plot(loss_train, label='train')
    plt.plot(loss_valid, label='val')
    plt.ylim(0)
    plt.legend()

    if len(metric_train) > 0 or len(metric_valid) > 0:
        plt.subplot(1, 2, 2)
        plt.plot(metric_train, label='train')
        plt.plot(metric_valid, label='val')
        plt.ylim(top=1)
        plt.legend()

In [ ]:
df = pd.read_csv("../data/interactions.csv")
df = df[['user_id', 'content_id', 'game', 'view']]
df.head()

In [ ]:
df_game = pd.read_csv("../data/articles.csv")
df_game.head()

In [ ]:
users_items_matrix_df = df.pivot(
    index='user_id',
    columns='content_id',
    values='view'
).fillna(0)

users_items_matrix_df.head()

In [ ]:
users_items_matrix_df.shape

# Model

In [ ]:
x = users_items_matrix_df.values

In [ ]:
model = Sequential([
    Dense(units=512, activation='selu', name='enc_1', input_dim=users_items_matrix_df.shape[1]),
    Dense(units=256, activation='selu', name='embeddings'),
    Dropout(rate=0.8, name='dropout'),
    Dense(units=512, activation='selu', name='dec_1'),
    Dense(units=users_items_matrix_df.shape[1], activation='linear', name='user_score_pred')
])
model.summary()

model.compile(optimizer=Adam(lr=1e-4), loss='mse')

hist = model.fit(x, x, epochs=50, batch_size=64, shuffle=True, validation_split=0.1)
plot_history(hist.history)

In [ ]:
new_matrix = model.predict(x) * (x == 0)

In [ ]:
new_users_items_matrix_df = pd.DataFrame(
    new_matrix,
    columns=users_items_matrix_df.columns,
    index=users_items_matrix_df.index
)
new_users_items_matrix_df.head()

In [ ]:
def recommender_for_user(user_id, interact_matrix, df_content, topn = 10):
    pred_scores = interact_matrix.loc[user_id].values

    df_scores = pd.DataFrame({
        'content_id': list(users_items_matrix_df.columns),
        'score': pred_scores
    })

    df_rec = (df_scores
        .set_index('content_id')
        .join(df_content.set_index('content_id'))
        .sort_values('score', ascending=False)
        .head(topn)[['score', 'game']]
    )

    return df_rec[df_rec.score > 0]

In [ ]:
# user_id = 1011, 1319
random_idx = np.random.choice(df.user_id.values, size=1)[0]
random_idx = 1011
recommender_for_user(user_id=random_idx, interact_matrix=users_items_matrix_df, df_content=df_game)

In [ ]:
recommender_for_user(user_id=random_idx, interact_matrix=new_users_items_matrix_df, df_content=df_game)

# Referências

- [Deep Learning para Sistemas de Recomendação (Parte 2) — Filtragem Colaborativa com AutoEncoders](https://medium.com/data-hackers/deep-learning-para-sistemas-de-recomendação-parte-2-filtragem-colaborativa-com-autoencoders-347ba7d53bae#:~:text=Um%20Autoencoder%20(AE)%20é%20uma,%2C%20f(x)%3Dh.)